In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
%run 'medcouple.ipynb'

## Model

In [3]:
def outliers_skewed(x):
    Q1 = x.quantile(0.25)
    Q3 = x.quantile(0.75)
    IQR = Q3 - Q1
    mc = medcouple_1d(x)
    if mc > 0:
        whisker_upper = 1.5*IQR*np.exp(3*mc)
        whisker_lower = 1.5*IQR*np.exp(-4*mc)
    else:
        whisker_upper = 1.5*IQR*np.exp(4*mc)
        whisker_lower = 1.5*IQR*np.exp(-3*mc)
    
    lower = Q1 - whisker_lower
    upper = Q3 + whisker_upper    
    return lower, upper 

In [4]:
def outlier_skewed_upper(x):
    Q1 = x.quantile(0.25)
    Q3 = x.quantile(0.75)
    IQR = Q3 - Q1
    mc = medcouple_1d(x)
    if mc > 0:
        whisker_upper = 1.5*IQR*np.exp(3*mc)
    else:
        whisker_upper = 1.5*IQR*np.exp(4*mc)
            
    upper = Q3 + whisker_upper    
    return upper

In [5]:
def mean_outliers_skewed(x):
    l, u = outliers_skewed(x)
    return x[(float(l)<=x)&(x<=float(u))].mean()

In [6]:
outliers_skewed(pd.Series([20, 16, 15, 15, 18, 22, 13]))

(13.41841717130564, 35.309690970754275)

In [7]:
def clicksPerAd(clicks, group_by=['ad_id', 'adsPerDisplay'], alpha=10, beta=0.1):

    res = pd.DataFrame({'clicked': clicks.groupby(group_by)['clicked'].sum(),
                         'viewed': clicks.groupby(group_by)['clicked'].size()
                          }).reset_index()
    
    res_mean = clicks['clicked'].mean()
   
    by = group_by[1:]
    if by<>[]:
        clicksPerAd_train_means = pd.DataFrame({'mean':clicks.groupby(by)['clicked'].mean()}).reset_index()
        res = res.merge(clicksPerAd_train_means, on=by)
        res['clicksPerShows'] = (beta*res_mean + alpha*res['mean'] + res['clicked']) / (beta + alpha + res['viewed'])
        res.drop(['clicked', 'viewed', 'mean'], axis=1, inplace=True)
    else:
        res['clicksPerShows'] = (alpha*res_mean + res['clicked']) / (alpha + res['viewed'])
        
    return res

In [8]:
def join(series):
    #return ' '.join( str(list(series)).strip('[]').split(', ') )
    return ' '.join( map(str, series) )

In [9]:
def model_prediction(clicksPerAd_train, test, on=['ad_id', 'adsPerDisplay']):
    prediction = pd.merge(test, clicksPerAd_train, how='left', on = on) 
    
    by = on[1:]
    clicksPerAd_train_means = pd.DataFrame({'clicksPerShows':clicksPerAd_train.groupby(by)['clicksPerShows'].mean()}).reset_index()
    
    index = pd.isnull(prediction['clicksPerShows'])
    prediction_nulls = prediction[index].copy()
    prediction_nulls.drop(['clicksPerShows'], axis=1, inplace=True)
    prediction_nulls = prediction_nulls.merge(clicksPerAd_train_means, how='left', on = by)#.fillna(clicksPerAd_train['clicksPerShows'].mean())
    prediction = pd.concat([prediction[~index], prediction_nulls])
    
    prediction = prediction.sort_values(by=['display_id', 'clicksPerShows'], ascending=[True, False])
    return pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).reset_index()

In [10]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

In [11]:
pd.read_csv("../generated/final/clicks_train_sample.csv", nrows=10).head()

display_id  document_id  timestamp  traffic_source  platform  day  hour  \
0           8      1330329        638             1.0       2.0    1     7   
1           8      1330329        638             1.0       2.0    1     7   
2           8      1330329        638             1.0       2.0    1     7   
3           8      1330329        638             1.0       2.0    1     7   
4        2657      1271490     185255             1.0       2.0    1     7   

    geo   ad_id  clicked  adsPerDisplay  clicksPerShows  
0  2765   95724        0              4        0.034785  
1  2765  175694        0              4        0.202327  
2  2765  280430        1              4        0.256853  
3  2765  329774        0              4        0.052426  
4  2765   70081        0              4        0.150145

In [26]:
group_by=['ad_id', 'adsPerDisplay']
usecols = ['display_id', 'ad_id', 'adsPerDisplay', 'clicked']

## Validation

In [13]:
clicks_train_sample = pd.read_csv("../generated/final/clicks_train_sample.csv", usecols = usecols)
clicks_train_sample.count()

display_id       65355241
ad_id            65355241
clicked          65355241
adsPerDisplay    65355241
dtype: int64

In [14]:
res = pd.DataFrame({'viewed': clicks_train_sample.groupby(['ad_id'])['clicked'].size()
                          }).reset_index()

res.head()

ad_id  viewed
0      1       2
1      2      16
2      3     114
3      4      26
4      6       1

In [16]:
print outliers_skewed(res['viewed'].copy())

(0.2346800274243106, 150.51359807572265)


In [17]:
res.count()

ad_id     447836
viewed    447836
dtype: int64

In [18]:
res = res[res['viewed']<151]
res.count()

ad_id     418258
viewed    418258
dtype: int64

In [19]:
res.drop(['viewed'], axis=1, inplace=True)
res.head()

ad_id
0      1
1      2
2      3
3      4
4      6

In [20]:
clicks_train_sample = clicks_train_sample.merge(res, on=['ad_id'])
clicks_train_sample.count()

display_id       5038425
ad_id            5038425
clicked          5038425
adsPerDisplay    5038425
dtype: int64

In [21]:
clicks_train_sample.head()

display_id  ad_id  clicked  adsPerDisplay
0        2657  70081        0              4
1     2035929  70081        0              4
2     7134381  70081        0              3
3      408154  70081        1              4
4     2013697  70081        0              4

In [22]:
clicksPerAd_train = clicksPerAd(clicks_train_sample,  group_by, alpha=10, beta=0.1)
clicksPerAd_train.count()

ad_id             1169028
adsPerDisplay     1169028
clicksPerShows    1169028
dtype: int64

In [23]:
clicks_test_sample = pd.read_csv("../generated/final/clicks_test_sample.csv", usecols = usecols)
clicks_test_sample.count()

display_id       21786490
ad_id            21786490
clicked          21786490
adsPerDisplay    21786490
dtype: int64

In [24]:
validation = clicks_test_sample[clicks_test_sample['clicked']==1].astype({'ad_id':str}).sort_values('display_id')
validation.count()

display_id       4220025
ad_id            4220025
clicked          4220025
adsPerDisplay    4220025
dtype: int64

In [25]:
validation.head()

display_id   ad_id  clicked  adsPerDisplay
18142714           1  144739        1              6
14412118           7  105766        1              3
11270842           9  140940        1              4
377269            14  224171        1              4
18233315          26  152193        1              2

In [27]:
prediction = model_prediction(clicksPerAd_train, clicks_test_sample, on=group_by)
prediction.head()

display_id                                     ad_id
0           1  42337 139684 144739 156824 279295 296965
1           7                      105766 215967 300808
2           9                19959 104208 140940 151028
3          14                98270 143467 224171 288396
4          26                             152193 285992

In [28]:
prediction.count()

display_id    4220025
ad_id         4220025
dtype: int64

In [29]:
print 'clicksPerShows :', model_evaluation(validation, prediction)  #0.496702629674

clicksPerShows : 0.490653182155


## Training

In [ ]:
print usecols
print group_by

In [ ]:
events_clicks_train = pd.read_csv("../generated/final/events_clicks_train.csv", usecols=usecols)
events_clicks_train.count()

In [ ]:
clicksPerAd_train = clicksPerAd(events_clicks_train,  group_by=group_by ,alpha=10, beta=0.1)
clicksPerAd_train.to_csv("../generated/solutions/7_clicksPerAd_train.csv", index=False)
clicksPerAd_train.count()

## Predicting

In [ ]:
print usecols
print group_by

In [ ]:
clicksPerAd_train = pd.read_csv("../generated/solutions/7_clicksPerAd_train.csv")

In [ ]:
clicks_test = pd.read_csv("../generated/final/events_clicks_test.csv", usecols=usecols[:-1])
clicks_test.count()

In [ ]:
prediction = model_prediction(clicksPerAd_train, clicks_test, on=group_by)
prediction.head()

In [ ]:
prediction.count()

Check order:

In [ ]:
ids = prediction.display_id
prev = 0
for nxt in ids:
    if nxt<prev:
        print nxt
        break
    prev = nxt 

## Submission

In [ ]:
prediction.to_csv("../submissions/solution_7.csv", index=False)